In [ ]:
from dgl.data import citation_graph,CoraDataset,TUDataset,GINDataset, ICEWS18
import torch
from dgl import DGLGraph
import numpy as np
import dgl
from datasets.prepocessing import one_class_processing

In [ ]:
data =citation_graph.load_cora()
#data =citation_graph.load_citeseer()
#data =TUDataset('PROTEINS_full')
#data =ICEWS18('train')

In [59]:
from dgl.data import tu
dataset = tu.TUDataset(name='PROTEINS_full')
train_size = int(0.6 * len(dataset))
test_size = int(0.2 * len(dataset))
val_size = int(len(dataset) - train_size - test_size)

dataset_train, dataset_val, dataset_test = torch.utils.data.random_split(
    dataset, (train_size, val_size, test_size))

In [60]:
dataset.graph_lists[0]

DGLGraph(num_nodes=42, num_edges=162,
         ndata_schemes={'node_labels': Scheme(shape=(), dtype=torch.int64), 'node_attr': Scheme(shape=(29,), dtype=torch.int64)}
         edata_schemes={})

In [61]:
g=dgl.transform.add_self_loop(dataset.graph_lists[0])
g.ndata.update(dataset.graph_lists[0].ndata)
dataset.graph_lists[0]=g
print(dataset.graph_lists[0])

DGLGraph(num_nodes=42, num_edges=204,
         ndata_schemes={'node_labels': Scheme(shape=(), dtype=torch.int64), 'node_attr': Scheme(shape=(29,), dtype=torch.int64)}
         edata_schemes={})


In [62]:
for i in range(len(dataset)):
    print(dataset.graph_lists[i])
    g=dgl.transform.add_self_loop(dataset.graph_lists[i])
    g.ndata.update(dataset.graph_lists[i].ndata)
    dataset.graph_lists[i]=g
    print(dataset.graph_lists[i])
    if i>0:
        break

DGLGraph(num_nodes=42, num_edges=204,
         ndata_schemes={'node_labels': Scheme(shape=(), dtype=torch.int64), 'node_attr': Scheme(shape=(29,), dtype=torch.int64)}
         edata_schemes={})
DGLGraph(num_nodes=42, num_edges=204,
         ndata_schemes={'node_labels': Scheme(shape=(), dtype=torch.int64), 'node_attr': Scheme(shape=(29,), dtype=torch.int64)}
         edata_schemes={})
DGLGraph(num_nodes=27, num_edges=92,
         ndata_schemes={'node_labels': Scheme(shape=(), dtype=torch.int64), 'node_attr': Scheme(shape=(29,), dtype=torch.int64)}
         edata_schemes={})
DGLGraph(num_nodes=27, num_edges=119,
         ndata_schemes={'node_labels': Scheme(shape=(), dtype=torch.int64), 'node_attr': Scheme(shape=(29,), dtype=torch.int64)}
         edata_schemes={})


In [ ]:
new_g.ndata.update(g.ndata)

In [ ]:
def batching_graph(batch):
    '''
    for dataset batching
    transform ndata to tensor (in gpu is available)
    '''
    graphs, labels = map(list, zip(*batch))
    #cuda = torch.cuda.is_available()

    # batch graphs and cast to PyTorch tensor
    for graph in graphs:
        for (key, value) in graph.ndata.items():
            graph.ndata[key] = value.float()
    batched_graphs = dgl.batch(graphs)

    # cast to PyTorch tensor
    batched_labels = torch.LongTensor(np.array(labels))

    return batched_graphs, batched_labels

In [ ]:
train_loader=torch.utils.data.DataLoader(dataset_train,
                                       batch_size=32,
                                       shuffle=True,
                                       collate_fn=batching_graph,
                                       drop_last=True,
                                       num_workers=4)

In [ ]:
for (batch_idx, (batch_graph, graph_labels)) in enumerate(train_loader):
    print(batch_graph)
    if batch_idx >3:
        break

In [ ]:
batch_graph

In [ ]:
batch_graph.

In [ ]:
input_dim, label_dim, max_num_node = dataset.statistics()

In [ ]:
print(len(dataset_train))
print(len(dataset_val))
print(len(dataset_test))

In [ ]:
label0=data[0][0].ndata['node_labels']
label1=data[1][0].ndata['node_labels']
label2=data[2][0].ndata['node_labels']

In [ ]:
label1

In [ ]:
print(len(label0))
print(len(label1))
print(len(label2))

In [ ]:
data[1][0].ndata['node_labels'].size()

In [ ]:
labels,train_mask,val_mask,test_mask=one_class_processing(data[0][0],4)

In [ ]:
from collections import Counter
Counter(data.labels)

In [ ]:
normal_class=2
labels,train_mask,val_mask,test_mask=one_class_processing(data.labels,normal_class)

In [ ]:
labels[train_mask.astype(int)].shape

In [ ]:
print('train',np.sum(labels[train_mask.astype(int)==1]))
print('val',np.sum(labels[val_mask.astype(int)==1]))
print('test',np.sum(labels[test_mask.astype(int)==1]))

In [ ]:
print('train',np.sum(train_mask))
print('val',np.sum(val_mask))
print('test',np.sum(test_mask))

In [ ]:
train_mask.shape